## 简介

目前最流行的两种集成算法分别为boosting和bagging。
- bagging ： 是bootstrap aggregatin（自助聚集）的简称。比如说，我们参加一次考试，但不知道该怎么做，这时候求助了几位朋友，对于数字型的问题，我们综合所有朋友的答案，求得均值；对于选择题，我们则将所有朋友的答案进行投票，选择票数最高的那个答案；

- boosting ： 即提升算法；这里同样也是求助朋友，只不过我们依据不同的问题，求助不同专业的朋友，比如对于音乐类的题，我们就求助音乐专业的同学，对于计算机问题则求助计算机专业的同学。

将如上的例子映射到机器学习中，那么这些“朋友“被称作”弱学习器“（并不表示模型是最弱的，只是相对于想要合成的强分类器而言较弱）。


### 为什么需要进行模型集成？

在训练最佳机器学习模型的过程中，有两个相悖的影响因素，即偏差（bias）与方差（variance），这经常会在面试中会被问到。

- Bias：当一个模型具有较高的bias时，这说明模型并不能很好的拟合数据（bending to data，也就是随着数据的变化而变化），如线性回归模型就是一个经常出现高bias的模型，如下图所示，它对于不同的数据都倾向于预测为同一条拟合线：
![amt64H.png](https://s1.ax1x.com/2020/07/29/amt64H.png)

Variance：当一个模型具有较高的Variance时，表示该模型为了满足数据中每一点的需求，导致预测值变化非常大。线性回归模型经常是一种低Variance高Bias的模型。一种具有高Variance低Bias的模型就是决策树（尤其是没有早停early stop的决策树），模型会尽可能地根据数据切分再切分。
![amaJUS.png](https://s1.ax1x.com/2020/07/29/amaJUS.png)

所以，采用多个子模型集成的方式，我们可以更好得在Bias与Variance中找到一个平衡的中间位置。当然，还有一些改进模型的其他方式，这些都是基于数学的最小化bias或者Variance的算法，比如中心极限定理。

## 随机森林

在进行模型集成以前，给高Variance的模型引入一定的随机变化，便可以降低Variance。主要可以通过如下两种方式：
- Boostrap the data：就是有放回的进行数据抽样，然后使用抽样后的数据进行模型拟合；
- subset the features：在决策树的每次分裂时或者在每个弱学习器中，使用特征集合的子集去拟合。

而使用如上方式的算法，便是随机森林（random forest）。

### 示例
决策树更倾向于去“记忆”数据，很容易产生过拟合；使用如上算法，选择多个特征集合的子集去训练多个决策树，最终将所有决策树的结果进行投票，得到预测结果。

## bagging算法

假设我们有一组如下的数据，数据量很大，所以，我们并不想按照上面的算法去使用全部数据训练多个模型，这样会耗费很大算力。
![amDxN6.png](https://s1.ax1x.com/2020/07/29/amDxN6.png)

这时候，我们可以有放回的对数据进行抽样（也就是说有可能有样本会取到多次，也有可能一次都没取到），然后用每个抽样后的数据去训练模型，最终将所有模型集成到一起，进行投票，得到最终结果。

<table>
<tr>
<td><img src=https://s1.ax1x.com/2020/07/29/amrKgg.png border=0></td>
<td><img src=https://s1.ax1x.com/2020/07/29/amroVI.png border=0></td>
<td><img src=https://s1.ax1x.com/2020/07/29/amsZL9.png border=0></td>
</tr>

<tr>
<td style="text-align:center">弱学习器A</td>
<td style="text-align:center">弱学习器B</td>
<td style="text-align:center">弱学习器C</td>
</tr>

</table>

三者合并，得到：

![amsgwn.png](https://s1.ax1x.com/2020/07/29/amsgwn.png)

## boosting算法

### 弱学习器的训练

Boosting提升算法相对来说较复杂一些，目前有很多基于boosting的算法，其中相对来说比较基础&流行的算法是Adaboosting算法。boosting算法的一个核心思想就是，提高分类错误样本的权重，这样再进行下一个弱学习器的拟合时，就会尽可能地避免将这类样本分错。

还是通过如上的示例，假设我们的三个弱学习器如下，从左至右，每次都是把错误的样本提升权重（这里用点的大小表示权重），最终将三个模型集成，并投票获得最终结果。

<table>
<tr>
<td><img src=https://s1.ax1x.com/2020/07/30/aMA4N6.png border=0></td>
<td><img src=https://s1.ax1x.com/2020/07/30/aMALDA.png border=0></td>
<td><img src=https://s1.ax1x.com/2020/07/30/aMEp8S.png border=0></td>
</tr>

<tr>
<td style="text-align:center">弱学习器A</td>
<td style="text-align:center">弱学习器B</td>
<td style="text-align:center">弱学习器C</td>
</tr>

</table>

**那么如何该提升多少权重呢？**

答案是将分类错误的权重设置为与分类正确的权重相等。我们可以将每一个样本的权重初始化，都为1；那么在第一幅图中，分类正确的权重为7，错误的权重为3，所以这里就给分类错误的样本权重乘以**7/3**，得到一个50/50的样本空间，然后再下一个学习器中，继续执行该操作。

### 集成弱学习器

经过前面的操作，我们获得了三个弱学习器，那么该如何将三者集成到一起呢？我们先来做个练习，假如说我们有三个朋友，一个朋友说的都是真话，一个说的有真有假，另外一个说的全是谎话，那么请问**这三个朋友中，哪一个对你最没有帮助？**
<img src="https://s1.ax1x.com/2020/07/30/aMZUnH.md.png"  width="500"/>

答案是：第二个朋友，也就是有真有假的那位。因为只说真话的朋友，我们正确得听建议就OK；只说谎话的朋友，我们反着听就OK；而有真有假的朋友，就很难判断何时该正着听，何时该反着听。

这在模型中也是一样，我们想要给正确率较高的弱学习器一个较大的正权重，给正确率较低的弱学习器一个绝对值较大的负权重，而正确率接近于0.5的模型分配权重为0.

那么对于如上的需求，我们可以使用如下函数来设置权重，其中$x$表示正确率，$y$表示权重。
<img src="https://s1.ax1x.com/2020/07/30/aMecPx.md.png"  width="500"/>

> 特殊情况：当弱分类器全部分类正确或者错误时，我们会得到一个正无穷或者负无穷的权重，这其实就表示该弱分类器已经完成了整个拟合任务，我们直接采用该模型就OK，就不需要再去计算其他模型了。（虽然这种情况几乎不可能出现）

那么，我们以此来计算前面的模型，最终得到的结果如下，那么结果的正负，就刚好可以表示出两个类别了：

<table>
<tr>
<td><img src=https://s1.ax1x.com/2020/07/30/aMmWYq.md.png border=0></td>
<td><img src=https://s1.ax1x.com/2020/07/30/aMmoXF.md.png border=0></td>
</tr>

</table>

### Sklearn中使用Adaboost

```python 
>>> from sklearn.ensemble import AdaBoostClassifier
>>> model = AdaBoostClassifier()
>>> model.fit(x_train, y_train)
>>> model.predict(x_test)

```

超参数：
```python

base_estimator: 弱学习器模型 (Warning: 别忘了导入弱学习器).
n_estimators: 弱学习器的最大数量.

```

示例：
```python
>>> from sklearn.tree import DecisionTreeClassifier
>>> model = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=2), n_estimators = 4)
```

### 拓展材料


- [The original paper](https://cseweb.ucsd.edu/~yfreund/papers/IntroToBoosting.pdf) - A link to the original paper on boosting by Yoav Freund and Robert E. Schapire.

- [An explanation about why boosting is so important](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/) - A great article on boosting by a Kaggle master, Ben Gorman.

- [A useful Quora post](https://www.quora.com/What-is-an-intuitive-explanation-of-Gradient-Boosting) - A number of useful explanations about boosting.

